<a href="https://colab.research.google.com/github/RigbeWeleslasie/Ankole-Information-Website/blob/main/BreastCncer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow opencv-python scikit-learn matplotlib split-folders

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import splitfolders
input_dir = "/content/drive/MyDrive/DAS/BUSI_Corrected"
output_dir = "/content/drive/MyDrive/DAS/BUSI_Corrected_split"
splitfolders.ratio(input_dir, output=output_dir, seed=1337, ratio=(.70, .15, .15))

ValueError: The provided input folder "/content/drive/MyDrive/DAS/BUSI_Corrected" does not exists.

In [ ]:
import os

input_dir_path = "/content/drive/MyDrive/DAS/BUSI_Corrected"

if os.path.exists(input_dir_path):
    print(f"The directory '{input_dir_path}' exists.")
else:
    print(f"The directory '{input_dir_path}' does not exist. Please verify the path in your Google Drive.")

The directory '/content/drive/MyDrive/DAS/BUSI_Corrected' does not exist. Please verify the path in your Google Drive.


In [ ]:
!pip install split-folders

In [ ]:
import splitfolders
input_dir = "/content/drive/MyDrive/DAS/BUSI_Corrected"
output_dir = "/content/drive/MyDrive/DAS/BUSI_Corrected_split"
splitfolders.ratio(input_dir, output=output_dir, seed=1337, ratio=(.70, .15, .15))

ValueError: The provided input folder "/content/drive/MyDrive/DAS/BUSI_Corrected" does not exists.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2, os, glob
import numpy as np
import tensorflow as tf
IMG_SIZE = (224, 224)
def load_with_opencv(filepath, label):
    img = cv2.imread(filepath, cv2.IMREAD_COLOR)  # always 3 channels
     # convert to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMG_SIZE)
    img = img.astype("float32") / 255.0
    return img, label
def load_files(root):
    classes = ["benign","malignant","normal"]
    filepaths, labels = [], []
    for cls in classes:
        for fp in glob.glob(os.path.join(root, cls, "*")):
            if fp.lower().endswith((".png",".jpg",".jpeg")):
                filepaths.append(fp)
                labels.append(1 if cls=="malignant" else 0)  # cancer=1
    return np.array(filepaths), np.array(labels)
train_fp, train_y = load_files("/content/drive/MyDrive/DAS/BUSI_Corrected_split/train")
val_fp,   val_y   = load_files("/content/drive/MyDrive/DAS/BUSI_Corrected_split/val")
test_fp,  test_y  = load_files("/content/drive/MyDrive/DAS/BUSI_Corrected_split/test")

In [ ]:
print(train_ds.element_spec)

(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))


In [ ]:
def make_dataset(filepaths, labels, training=False):
    imgs = []
    for i, fp in enumerate(filepaths):
        img, _ = load_with_opencv(fp, labels[i])
        imgs.append(img)
    X = np.array(imgs, dtype="float32")
    y = np.array(labels, dtype="float32")
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if training:
        ds = ds.shuffle(buffer_size=1024) # Use a fixed buffer size for shuffling
    ds = ds.batch(32).prefetch(tf.data.AUTOTUNE)
    return ds
train_ds = make_dataset(train_fp, train_y, training=True)
val_ds   = make_dataset(val_fp, val_y)
test_ds  = make_dataset(test_fp, test_y)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow as tf


def preprocess_image(image, label):

    image = tf.image.grayscale_to_rgb(image) if image.shape[-1] == 1 else image
    return image, label

train_dir = '/content/drive/MyDrive/DAS/BUSI_Corrected_split/train'
val_dir = '/content/drive/MyDrive/DAS/BUSI_Corrected_split/val'


train_ds_initial = image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=32,
    color_mode='grayscale',
    label_mode='int',
    class_names=['benign', 'malignant', 'normal']
)

val_ds_initial = image_dataset_from_directory(
     val_dir,
    image_size=IMG_SIZE,
    batch_size=32,
    color_mode='grayscale',
    label_mode='int', #
    class_names=['benign', 'malignant', 'normal'] )


train_ds = train_ds.map(preprocess_image)
val_ds = val_ds.map(preprocess_image)

Found 524 files belonging to 3 classes.
Found 111 files belonging to 3 classes.


In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax') # Output layer with 3 units for 3 classes and softmax activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', # Use sparse_categorical_crossentropy for integer labels
              metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
epochs = 10  # You can adjust the number of epochs
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,347 (42.61 MB)

 Trainable params: 11,169,347 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.6084 - loss: 0.9859 - val_accuracy: 0.7658 - val_loss: 0.5685
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.7924 - loss: 0.4698 - val_accuracy: 0.8559 - val_loss: 0.3852
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.8459 - loss: 0.3591 - val_accuracy: 0.8378 - val_loss: 0.3063
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.8697 - loss: 0.3085 - val_accuracy: 0.8649 - val_loss: 0.2762
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.9056 - loss: 0.2417 - val_accuracy: 0.8739 - val_loss: 0.2762
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.9164 - loss: 0.1950 - val_accuracy: 0.8919 - val_loss: 0.2100
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.9713 - loss: 0.0981 - val_accuracy: 0.9099 - val_loss: 0.2405
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.9666 - loss: 0.0919 - val_accuracy: 0.8919 - val_loss:

In [ ]:

loss, accuracy = model.evaluate(test_ds)

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 475ms/step - accuracy: 0.8961 - loss: 0.7967
Test Loss: 1.1610462665557861
Test Accuracy: 0.852173924446106


In [ ]:
        !pip install vscode_colab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.5 MB/s eta 0:00:00


In [ ]:
        import vscode_colab
        vscode_colab.login()


2025-10-05 14:55:06 | INFO     | vscode_colab.server:download_vscode_cli:74 - Downloading VS Code CLI (cli-alpine-x64) to /content/vscode_cli_alpine_x64.tar.gz...
2025-10-05 14:55:07 | INFO     | vscode_colab.server:download_vscode_cli:87 - VS Code CLI tarball downloaded. Extracting...
2025-10-05 14:55:08 | INFO     | vscode_colab.server:download_vscode_cli:171 - VS Code CLI setup successful. Executable at: '/content/code'.
2025-10-05 14:55:08 | INFO     | vscode_colab.server:_login:220 - Initiating VS Code Tunnel login with command: /content/code tunnel user login --provider github
2025-10-05 14:55:08 | INFO     | vscode_colab.server:_login:254 - Monitoring login process output for GitHub authentication URL and code...
2025-10-05 14:55:09 | INFO     | vscode_colab.server:_login:272 - Detected authentication URL: https://github.com/login/device
2025-10-05 14:55:09 | INFO     | vscode_colab.server:_login:278 - Detected authentication code: 6051-6A24
2025-10-05 14:55:09 | INFO     | vsco

2025-10-05 14:55:09 | INFO     | vscode_colab.server:login:200 - Login successful: Set environment variable VSCODE_COLAB_LOGGED_IN=true


True

In [ ]:
        vscode_colab.connect(name="my-tunnel", git_user_name="RigbeWeleslasie", git_user_email="rigbewweleslasie@gmail.com")

2025-10-05 14:56:35 | INFO     | vscode_colab.server:connect:579 - Initial CWD for connect operation: /content
2025-10-05 14:56:35 | INFO     | vscode_colab.server:download_vscode_cli:58 - VS Code CLI already exists and is executable at /content/code. Skipping download.
2025-10-05 14:56:35 | INFO     | vscode_colab.environment.git_handler:configure_git:27 - Attempting to set git global user.name='RigbeWeleslasie' and user.email='rigbewweleslasie@gmail.com'...
2025-10-05 14:56:35 | INFO     | vscode_colab.environment.git_handler:configure_git:53 - Successfully set git global user.name='RigbeWeleslasie'.
2025-10-05 14:56:35 | INFO     | vscode_colab.environment.git_handler:configure_git:88 - Successfully set git global user.email='rigbewweleslasie@gmail.com'.
2025-10-05 14:56:35 | INFO     | vscode_colab.server:_launch_and_monitor_tunnel:457 - Starting VS Code tunnel with command: /content/code tunnel --accept-server-license-terms --name my-tunnel --install-extension mgesbert.python-path